# LangChain Expression Language(LCEL)
다양한 구성 요소를 단일 체인으로 결합  
https://wikidocs.net/233344

```
chain = prompt | model | output_parser
```

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

# 1
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

# 2
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

# 4
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = ChatPromptTemplate.from_template(template)


##########
# llm 모델
##########
local_path = (
    "./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"
)
llm = GPT4All(
    model=local_path,
    callbacks=[StreamingStdOutCallbackHandler()],
    backend="mps", # GPU 설정
    streaming=True,
    verbose=True,
)


chain = (
    prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))


영어 대화 예시:

English Conversation:
I would like to order some food at the restaurant, please.
Can I have a menu?
Sure, here you go. Please take your time and look through it. If you need any help or recommendations, just let me know.
Thank you very much! I'll take a moment to decide what I want.
Great, we appreciate your business. Let us know when you're ready to place your order.
I think I found something that looks good. Can I have the [specific dish] please?
Sure thing! Would you like anything else with it or any side dishes?
No thank you, just the main dish is fine for now.
Alright then, we will get right on that and bring it out to your table shortly. Enjoy your meal!
영어 대화 예시:

English Conversation:
I would like to order some food at the restaurant, please.
Can I have a menu?
Sure, here you go. Please take your time and look through it. If you need any help or recommendations, just let me know.
Thank you very much! I'll take a moment to decide what I want.
Great, we appreciate you

# JSONPatch 청크 스트리밍

https://wikidocs.net/233345

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
##########
# llm 모델
##########
local_path = (
    "./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"
)
llm = GPT4All(
    model=local_path,
    callbacks=[StreamingStdOutCallbackHandler()],
    backend="mps", # GPU 설정
    streaming=True,
    verbose=True,
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
#################
# embedding model
#################
model_name = 'BM-K/KoSimCSE-roberta'
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True}

embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
###################################
#  FAISS 벡터 스토어를 이용해 텍스트를 인덱싱
###################################
vectorstore = FAISS.from_texts(
    ["원영이가 살고 있는 곳은 대한민국 입니다."], embedding=embeddings_model
)
#########################################################################
# retriever와 RunnablePassthrough를 통해 질문의 컨텍스트를 검색하고, 질문을 그대로 전달
# retrieval_chain은 이러한 컴포넌트들을 파이프라인으로 연결하고, StrOutputParser를 사용하여 모델의 출력을 문자열로 파싱합니다.
#########################################################################
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다
async for chunk in retrieval_chain.astream_log(
    "원영이가 살고 있는 곳은 어딘가요?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with MEAN pooling.
/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '65842086-4c8f-4148-9f4c-0d58101b8f59',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'ecfbd609-acec-4293-80ea-609f46f99ad9',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-05-14T13:39:57.127+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'HuggingFaceBgeEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='원영이가 살고 있는 곳은 대한민국 입니다.')

`stream`: 응답의 청크를 스트리밍합니다  
`invoke`: 입력에 대해 체인을 호출합니다  
`batch`: 입력 목록에 대해 체인을 호출합니다  

In [ ]:
chain.invoke({"topic": "ChatGPT"})



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

'\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.'

In [ ]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출

chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.

['\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.']

In [ ]:
# 최대 3개의 작업을 동시에 처리

chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.
멀티모달은 여러 모드를 통해 정보를 전달하는 것을 말합니다. 이는 시각, 청각 또는 촉각을 포함할 수 있습니다. 예를 들어, 비디오는 시청자에게 동시에 이미지(시각)와 소리(청각)를 제공합니다. 마찬가지로, 만져볼 수 있는 물건들은 촉감을 통한 정보 제공에 도움을 줍니다. 멀티모달은 사람들이 정보를 더 잘 이해하고 기억하는데 도움이 될 수 있으며, 다양한 감각 채널을 통해 전달되기 때문입니다.
프로그래밍은 복잡한 작업을 수행하기 위해 기계를 지시하는 일련의 명령어들을 작성하는

['\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.',
 '\n멀티모달은 여러 모드를 통해 정보를 전달하는 것을 말합니다. 이는 시각, 청각 또는 촉각을 포함할 수 있습니다. 예를 들어, 비디오는 시청자에게 동시에 이미지(시각)와 소리(청각)를 제공합니다. 마찬가지로, 만져볼 수 있는 물건들은 촉감을 통한 정보 제공에 도움을 줍니다. 멀티모달은 사람들이 정보를 더 잘 이해하고 기억하는데 도움이 될 수 있으며, 다양한 감각 채널을 통해 전달되기 때문입니다.',
 '\n프로그래밍은 복잡한 작업을 수행하기 

`astream`: 비동기적으로 응답의 청크를 스트리밍합니다  
`ainvoke`: 비동기적으로 입력에 대해 체인을 호출합니다  
`abatch`: 비동기적으로 입력 목록에 대해 체인을 호출합니다  
`astream_log`: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다

In [ ]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for s in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(s.content, end="", flush=True)


YouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.

AttributeError: 'str' object has no attribute 'content'

chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다

In [ ]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
await chain.ainvoke({"topic": "NVDA"})



1. NonVisual Desktop Access, 즉 NVDA는 시각 장애가 있는 개인들이 화면 내용을 읽고 이해할 수 있도록 도와주는 무료 소프트웨어 응용 프로그램입니다.
2. 이 접근성 도구로 사용자들은 키보드 단축키와 음성 합성을 통해 웹사이트, 문서 및 기타 디지털 콘텐츠를 효과적으로 상호 작용할 수 있습니다.
3. NVDA는 다양한 운영 체제에서 사용할 수 있으며 지속적으로 업데이트되고 개선되어 시각 장애가 있는 개인들이 정보에 접근하고 기술을 활용할 수 있도록 보장합니다.

'\n\n1. NonVisual Desktop Access, 즉 NVDA는 시각 장애가 있는 개인들이 화면 내용을 읽고 이해할 수 있도록 도와주는 무료 소프트웨어 응용 프로그램입니다.\n2. 이 접근성 도구로 사용자들은 키보드 단축키와 음성 합성을 통해 웹사이트, 문서 및 기타 디지털 콘텐츠를 효과적으로 상호 작용할 수 있습니다.\n3. NVDA는 다양한 운영 체제에서 사용할 수 있으며 지속적으로 업데이트되고 개선되어 시각 장애가 있는 개인들이 정보에 접근하고 기술을 활용할 수 있도록 보장합니다.'

In [ ]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
await chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)


YouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.
Facebook은 전 세계 수백만 명의 사람들이 친구, 가족 및 관심사와 연결될 수 있는 소셜 네트워킹 플랫폼입니다. 사용자들은 자신의 프로필을 만들고 업데이트를 게시하며 다른 사람들과 상호 작용할 수 있습니다. 또한 페이스북 메신저와 인스타그램과 같은 다양한 기능을 제공하여 사용자가 서로 소통하고 콘텐츠를 공유할 수 있게 합니다.

Facebook은 2004년 하버드 대학교 학생인 마크 주커버그에 의해 창립되었습니다. 처음에는 학교 내의 학생들만을 대상으로 했지만, 이후 전 세계 수백만 명의 사용자를 보유한 글로벌 플랫폼으로 성장했습니다. Facebook은 사용자 데이터를 수집하고 이를 기반으로 타겟 광고를 제공하는 것으로 알려져 있으며, 이는 회사의 주요 수익원입니다.

Facebook은 개인정보 보호 문제와 잘못된 정보 확산에 대한 비판을 받아왔으며, 이로 인해 많은 논란이 일고 있습니다. 그럼에도 불구하고 전 세계적으로 가장 인기 있는 소셜 미디어 플랫폼 중 하나로 남아있으며, 사람들이 연결되고 정보를 공유하는 데 있어 중요한 역할을 하고 있습니다.

['\nYouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.',
 '\nFacebook은 전 세계 수백만 명의 사람들이 친구, 가족 및 관심사와 연결될 수 있는 소셜 네트워킹 플랫폼입니다. 사용자들은 자신의 프로필을 만들고 업데이트를 게시하며 다른 사람들과 상호 작용할 수 있습니다. 또한 페이스북 메신저와 인스타그램과 같은 다양한 기능을 제공하여 사용자가 서로 소통하고 콘텐츠를 공유할 수 있게 합니다.\n\nFacebook은 2004년 하버드 대학교 학생인 마크 주커버그에 의해 창립되었습니다. 처음에는 학교 내의 학생들만을 대상으로 했지만, 이후 전 세계 수백만 명의 사용자를 보유한 글로벌 플랫폼으로 성장했습니다. Facebook은 사용자 데이터를 수집하고 이를 기반으로 타겟 광고를 제공하는 것으로 알려져 있으며, 이는 회사의 주요 수익원입니다.\n\nFacebook은 개인정보 보호 문제와 잘못된 정보 확산에 대한 비판을 받아왔으며, 이로 인해 많은 논란이 일고 있습니다. 그럼에도 불구하고 전 세계적으로 가장 인기 있는 소셜 미디어 플랫폼 중 하나로 남아있으며, 사람들이 연결되고 정보를 공유하는 데 있어 중요한 역할을 하고 있습니다.']

# from llama_cpp import Llama 챗봇

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from llama_cpp import Llama

llm = Llama(model_path="./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf")

question = "대한민국의 수도는?"

result = llm.create_chat_completion(
        
        messages=[{
            "role": "user",
            "content": question
    }],
    temperature=0.1,
    max_tokens=2048,
)

print(result)

llama_model_loader: loaded meta data with 25 key-value pairs and 435 tensors from ./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = 01-original_model
llama_model_loader: - kv   2:                          llama.block_count u32              = 48
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: 

{'id': 'chatcmpl-5b858b5e-4261-47f0-99ea-26ac2e72f214', 'object': 'chat.completion', 'created': 1715677476, 'model': './EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '대한민국의 수도는 서울입니다. 서울은 나라의 정치, 경제, 문화 중심지로서 중요한 역할을 하고 있습니다. 약 1000만 명이 거주하는 대도시로, 다양한 역사 유적지와 현대적인 건축물들이 어우러져 있어 방문객들에게 풍부한 경험을 제공합니다.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 27, 'completion_tokens': 61, 'total_tokens': 88}}


In [28]:
result['choices'][0]['message']['content']

'대한민국의 수도는 서울입니다. 서울은 나라의 정치, 경제, 문화 중심지로서 중요한 역할을 하고 있습니다. 약 1000만 명이 거주하는 대도시로, 다양한 역사 유적지와 현대적인 건축물들이 어우러져 있어 방문객들에게 풍부한 경험을 제공합니다.'

# from langchain_community.llms import GPT4All 챗봇

In [29]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt


PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

In [33]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

local_path = (
    "./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"
)

llm = GPT4All(
    model=local_path,
    callbacks=[StreamingStdOutCallbackHandler()],
    # backend="gpu", # GPU 설정
    streaming=True,
    verbose=True,
)
#####################################
# LCEL(LangChain Expression Language)
# prompt 템플릿과 모델을 함께 연결
#####################################
chain = prompt | llm | StrOutputParser()

# 질의 실행
response = chain.invoke({"country": "Smart phone"})


 - ITWorld Korea
스마트폰이 얼마나 되는지 알고 싶으세요? 이 질문에 대한 답은 여러 가지가 있습니다. 하지만, 가장 정확한 대답을 원한다면 IDC(International Data Corporation)를 참고하세요.
IDC에 따르면 전 세계적으로 약 10억 대의 스마트폰을 사용 중이라고 합니다. 이는 지난해 같은 기간보다 무려 42% 증가한 수치입니다. 이 수치는 올해 말까지 35% 더 성장할 것으로 예상됩니다. IDC는 연말까지 총 13억 6,800만대의 기기가 판매될 것이라고 예측하고 있습니다.
IDC의 모바일 폰 부문 책임자인 라몬 로페즈(Ramon Lopez)에 따르면 "스마트폰은 이제 소비자 시장에서 가장 빠르게 성장하는 카테고리 중 하나가 되었습니다."라고 합니다. 그는 또한 스마트폰을 사용하는 사람들의 수가 2015년까지 전 세계적으로 약 3억 명으로 증가할 것으로 예상하고 있습니다.
이 수치는 IDC가 지난달에 발표한 자료와 일치합니다. 당시에는 올해 말까지 총 9억 대의 기기가 판매될 것이라고 예측했었습니다. 하지만, 로페즈는

In [37]:
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))


개요:
식당에 가기 위해서는 먼저 식당을 선택해야 합니다. 그 다음에는 메뉴를 살펴보고 주문을 할 수 있는 테이블이나 카운터로 이동합니다. 원하는 음식을 선택한 후, 종업원에게 주문하고 싶은 음식과 음료의 종류를 알려줍니다. 그들은 당신의 주문을 받아서 준비하며, 식사를 즐길 준비를 하면서 기다립니다.

예시:
식당에 가기 위해서는 먼저 식당을 선택해야 합니다. 예를 들어 '맛있는 식당'이라고 해봅시다. 그 다음에는 메뉴를 살펴보고 주문을 할 수 있는 테이블이나 카운터로 이동합니다. 맛있는 치즈버거와 콜라를 먹고 싶다고 결정했습니다. 종업원에게 다가가서 "치즈버거 하나랑 콜라도 주세요."라고 말씀드립니다. 그들은 당신의 주문을 받아서 준비하며, 식사를 즐길 준비를 하면서 기다립니다.

출처:
이 정보는 식당에서의 일반적인 식사 경험을 바탕으로 합니다.
개요:
식당에 가기 위해서는 먼저 식당을 선택해야 합니다. 그 다음에는 메뉴를 살펴보고 주문을 할 수 있는 테이블이나 카운터로 이동합니다. 원하는 음식을 선택한 후, 종업원에게 주문하고 싶은 음식과 음료의 종류를 알려줍니다. 그들은 당신의 주문을 받아서 준비하며, 식사를 즐길 준비를 하면서 기다립니다.

예시:
식당에 가기 위해서는 먼저 식당을 선택해야 합니다. 예를 들어 '맛있는 식당'이라고 해봅시다. 그 다음에는 메뉴를 살펴보고 주문을 할 수 있는 테이블이나 카운터로 이동합니다. 맛있는 치즈버거와 콜라를 먹고 싶다고 결정했습니다. 종업원에게 다가가서 "치즈버거 하나랑 콜라도 주세요."라고 말씀드립니다. 그들은 당신의 주문을 받아서 준비하며, 식사를 즐길 준비를 하면서 기다립니다.

출처:
이 정보는 식당에서의 일반적인 식사 경험을 바탕으로 합니다.


In [39]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:

- 한글 해석:
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))


영어 대화:
"Excuse me, sir/ma'am, I would like to place an order."

번역:
저에게 잠시 시간을 내어주실 수 있을까요? 주문을 하고 싶습니다.
영어 대화:
"Excuse me, sir/ma'am, I would like to place an order."

번역:
저에게 잠시 시간을 내어주실 수 있을까요? 주문을 하고 싶습니다.


In [44]:
chain

PromptTemplate(input_variables=['question'], template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.\n\n상황:\n{question}\n\nFORMAT:\n- 영어 회화:\n- 한글 해석:\n')
| GPT4All(verbose=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x16a13a290>], model='./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf', streaming=True, client=<gpt4all.gpt4all.GPT4All object at 0x16a139ed0>)
| StrOutputParser()

In [54]:
len(dir(chain)[78:])

57

In [40]:
print(chain.invoke({"question": "미국에서 피자 주문"}))


영어 대화:
John: 안녕하세요, Pizza Hut에 전화드렸습니다. 한 판의 슈퍼 사워 치즈 페페로니 피자를 주세요. 그리고 사이드로 감자튀김도 같이 보내주세요. 감사합니다!

번역: 존은 피자헛에 전화를 걸어 슈퍼 소어 체이스 페페로니 피자와 함께 감자튀김을 주문했습니다. 그는 주문을 마치고 '감사합니다'라고 말했습니다.
영어 대화:
John: 안녕하세요, Pizza Hut에 전화드렸습니다. 한 판의 슈퍼 사워 치즈 페페로니 피자를 주세요. 그리고 사이드로 감자튀김도 같이 보내주세요. 감사합니다!

번역: 존은 피자헛에 전화를 걸어 슈퍼 소어 체이스 페페로니 피자와 함께 감자튀김을 주문했습니다. 그는 주문을 마치고 '감사합니다'라고 말했습니다.


In [42]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
(prompt | llm).invoke(input)


양자 역학은 원자의 행동을 지배하는 물리학의 한 분야입니다. 이 작은 입자들은 우리가 일상 생활에서 보는 세계와는 매우 다르게 작동합니다. 예를 들어, 그들은 동시에 여러 곳에 존재할 수 있으며(슈퍼포지션), 그들의 위치와 운동량을 정확히 측정할 수는 없습니다(불확정성 원칙).
양자 역학을 이해하는 것은 도전적일 수 있지만, 그 원리를 탐구하는 것이 흥미롭습니다. 이 분야는 물리학에서 가장 활발한 분야 중 하나이며 새로운 발견이 지속적으로 이루어지고 있습니다. 양자 역학은 또한 현대 기술 발전에 필수적인 역할을 했으며, 예를 들어 트랜지스터와 레이저 등이 그것입니다.
양자 역학의 기본 원리를 이해하기 위해 다음과 같은 몇 가지 핵심 개념을 살펴보겠습니다:
* 입자의 파동-입자 이중성: 양자 세계에서 모든 것은 동시에 파동과 입자로 존재합니다. 이 개념은 아인슈타인의 광전 효과에 대한 설명에서 처음 소개되었습니다. 그는 빛이 전자기파의 형태로 방출되지만, 특정 조건 하에서는 개별 입자인 광자로 행동한다고 제안했습니다.
* 슈퍼포지션: 양자 세계에서 입자들은 동시에 여러 곳에 존재할 수 있습니다.

'\n양자 역학은 원자의 행동을 지배하는 물리학의 한 분야입니다. 이 작은 입자들은 우리가 일상 생활에서 보는 세계와는 매우 다르게 작동합니다. 예를 들어, 그들은 동시에 여러 곳에 존재할 수 있으며(슈퍼포지션), 그들의 위치와 운동량을 정확히 측정할 수는 없습니다(불확정성 원칙).\n양자 역학을 이해하는 것은 도전적일 수 있지만, 그 원리를 탐구하는 것이 흥미롭습니다. 이 분야는 물리학에서 가장 활발한 분야 중 하나이며 새로운 발견이 지속적으로 이루어지고 있습니다. 양자 역학은 또한 현대 기술 발전에 필수적인 역할을 했으며, 예를 들어 트랜지스터와 레이저 등이 그것입니다.\n양자 역학의 기본 원리를 이해하기 위해 다음과 같은 몇 가지 핵심 개념을 살펴보겠습니다:\n* 입자의 파동-입자 이중성: 양자 세계에서 모든 것은 동시에 파동과 입자로 존재합니다. 이 개념은 아인슈타인의 광전 효과에 대한 설명에서 처음 소개되었습니다. 그는 빛이 전자기파의 형태로 방출되지만, 특정 조건 하에서는 개별 입자인 광자로 행동한다고 제안했습니다.\n* 슈퍼포지션: 양자 세계에서 입자들은 동시에 여러 곳에 존재할 수 있습니다.'

In [56]:
# parse_output 메서드를 사용하여 AI 모델이 생성한 메시지 문자열로 출력합니다.
(prompt | llm | StrOutputParser()).invoke(input)


양자 역학은 원자의 행동을 지배하는 물리학의 한 분야입니다. 이 작은 입자들은 우리가 일상 생활에서 보는 세계와는 매우 다르게 작동합니다. 예를 들어, 그들은 동시에 여러 곳에 존재할 수 있으며(슈퍼포지션), 그들의 위치와 운동량을 정확히 측정할 수는 없습니다(불확정성 원칙).
양자 역학을 이해하는 것은 도전적일 수 있지만, 그 원리를 탐구하는 것이 흥미롭습니다. 이 분야는 물리학에서 가장 활발한 분야 중 하나이며 새로운 발견이 지속적으로 이루어지고 있습니다. 양자 역학은 또한 현대 기술 발전에 필수적인 역할을 했으며, 예를 들어 트랜지스터와 레이저 등이 그것입니다.
양자 역학의 기본 원리를 이해하기 위해 다음과 같은 몇 가지 핵심 개념을 살펴보겠습니다:
* 입자의 파동-입자 이중성: 양자 세계에서 모든 것은 동시에 파동과 입자로 존재합니다. 이 개념은 아인슈타인의 광전 효과에 대한 설명에서 처음 소개되었습니다. 그는 빛이 전자기파의 형태로 방출되지만, 특정 조건 하에서는 개별 입자인 광자로 행동한다고 제안했습니다.
* 슈퍼포지션: 양자 세계에서 입자들은 동시에 여러 곳에 존재할 수 있습니다.

'\n양자 역학은 원자의 행동을 지배하는 물리학의 한 분야입니다. 이 작은 입자들은 우리가 일상 생활에서 보는 세계와는 매우 다르게 작동합니다. 예를 들어, 그들은 동시에 여러 곳에 존재할 수 있으며(슈퍼포지션), 그들의 위치와 운동량을 정확히 측정할 수는 없습니다(불확정성 원칙).\n양자 역학을 이해하는 것은 도전적일 수 있지만, 그 원리를 탐구하는 것이 흥미롭습니다. 이 분야는 물리학에서 가장 활발한 분야 중 하나이며 새로운 발견이 지속적으로 이루어지고 있습니다. 양자 역학은 또한 현대 기술 발전에 필수적인 역할을 했으며, 예를 들어 트랜지스터와 레이저 등이 그것입니다.\n양자 역학의 기본 원리를 이해하기 위해 다음과 같은 몇 가지 핵심 개념을 살펴보겠습니다:\n* 입자의 파동-입자 이중성: 양자 세계에서 모든 것은 동시에 파동과 입자로 존재합니다. 이 개념은 아인슈타인의 광전 효과에 대한 설명에서 처음 소개되었습니다. 그는 빛이 전자기파의 형태로 방출되지만, 특정 조건 하에서는 개별 입자인 광자로 행동한다고 제안했습니다.\n* 슈퍼포지션: 양자 세계에서 입자들은 동시에 여러 곳에 존재할 수 있습니다.'

In [57]:
from langchain.prompts import ChatPromptTemplate

# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | llm

In [58]:
print(chain)

first=ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='{topic} 에 대하여 3문장으로 설명해줘.'))]) last=GPT4All(verbose=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x16a13a290>], model='./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf', streaming=True, client=<gpt4all.gpt4all.GPT4All object at 0x16a139ed0>)


In [59]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [60]:
chain.output_schema.schema()

{'title': 'GPT4AllOutput', 'type': 'string'}

In [61]:
for s in chain.stream({"topic": "멀티모달"}):
    print(s.content, end="", flush=True)



멀티모달은 여러 모드를 통해 정보를 전달하는 것을 말합니다. 이는 시각, 청각 또는 촉각을 포함할 수 있습니다. 예를 들어, 비디오는 시청자에게 동시에 이미지(시각)와 소리(청각)를 제공합니다. 마찬가지로, 만져볼 수 있는 물건들은 촉감을 통한 정보 제공에 도움을 줍니다. 멀티모달은 사람들이 정보를 더 잘 이해하고 기억하는데 도움이 될 수 있으며, 다양한 감각 채널을 통해 전달되기 때문입니다.

AttributeError: 'str' object has no attribute 'content'

In [64]:
chain.invoke({"topic": "ChatGPT"})



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

'\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.'

In [65]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출

chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.

['\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.']

In [68]:
# 최대 3개의 작업을 동시에 처리

chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)



ChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.

ChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.
멀티모달은 여러 모드를 통해 정보를 전달하는 것을 말합니다. 이는 시각, 청각 또는 촉각을 포함할 수 있습니다. 예를 들어, 비디오는 시청자에게 동시에 이미지(시각)와 소리(청각)를 제공합니다. 마찬가지로, 만져볼 수 있는 물건들은 촉감을 통한 정보 제공에 도움을 줍니다. 멀티모달은 사람들이 정보를 더 잘 이해하고 기억하는데 도움이 될 수 있으며, 다양한 감각 채널을 통해 전달되기 때문입니다.
프로그래밍은 복잡한 작업을 수행하기 위해 기계를 지시하는 일련의 명령어들을 작성하는

['\n\nChatGPT는 OpenAI에서 개발한 대화형 언어 모델로, 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계되었습니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.\n\nChatGPT는 대규모의 훈련 데이터와 자연어 처리 기술을 활용하여 인간과 같은 방식으로 상호작용하고 응답할 수 있도록 설계된 대화형 언어 모델입니다. 이 AI 기반 시스템은 다양한 주제에 대해 정보를 제공하고, 질문에 답하며, 대화를 나눌 수 있어 교육, 고객 지원 및 기타 여러 분야에서 유용하게 활용되고 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.',
 '\n멀티모달은 여러 모드를 통해 정보를 전달하는 것을 말합니다. 이는 시각, 청각 또는 촉각을 포함할 수 있습니다. 예를 들어, 비디오는 시청자에게 동시에 이미지(시각)와 소리(청각)를 제공합니다. 마찬가지로, 만져볼 수 있는 물건들은 촉감을 통한 정보 제공에 도움을 줍니다. 멀티모달은 사람들이 정보를 더 잘 이해하고 기억하는데 도움이 될 수 있으며, 다양한 감각 채널을 통해 전달되기 때문입니다.',
 '\n프로그래밍은 복잡한 작업을 수행하기 

비동기 for 루프(async for)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, print 함수를 통해 메시지의 내용(s.content)을 즉시 출력합니다. end=""는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.

In [69]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for s in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(s.content, end="", flush=True)


YouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.

AttributeError: 'str' object has no attribute 'content'

chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다

In [67]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
await chain.ainvoke({"topic": "NVDA"})



1. NonVisual Desktop Access, 즉 NVDA는 시각 장애가 있는 개인들이 화면 내용을 읽고 이해할 수 있도록 도와주는 무료 소프트웨어 응용 프로그램입니다.
2. 이 접근성 도구로 사용자들은 키보드 단축키와 음성 합성을 통해 웹사이트, 문서 및 기타 디지털 콘텐츠를 효과적으로 상호 작용할 수 있습니다.
3. NVDA는 다양한 운영 체제에서 사용할 수 있으며 지속적으로 업데이트되고 개선되어 시각 장애가 있는 개인들이 정보에 접근하고 기술을 활용할 수 있도록 보장합니다.

'\n\n1. NonVisual Desktop Access, 즉 NVDA는 시각 장애가 있는 개인들이 화면 내용을 읽고 이해할 수 있도록 도와주는 무료 소프트웨어 응용 프로그램입니다.\n2. 이 접근성 도구로 사용자들은 키보드 단축키와 음성 합성을 통해 웹사이트, 문서 및 기타 디지털 콘텐츠를 효과적으로 상호 작용할 수 있습니다.\n3. NVDA는 다양한 운영 체제에서 사용할 수 있으며 지속적으로 업데이트되고 개선되어 시각 장애가 있는 개인들이 정보에 접근하고 기술을 활용할 수 있도록 보장합니다.'

In [66]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
await chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)


YouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.
인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.
Facebook은 전 세계 수백만 명의 사람들이 친구, 가족 및 관심사와 연결될 수 있는 소셜 네트워킹 플랫폼입니다. 사용자들은 자신의 프로필을 만들고 업데이트를 게시하며 다른 사람들과 상호 작용할 수 있습니다. 또한 페이스북 메신저와 인스타그램과 같은 다양한 기능을 제공하여 사용자가 서로 소통하고 콘텐츠를 공유할 수 있게 합니다.

Facebook은 2004년 하버드 대학교 학생인 마크 주커버그에 의해 창립되었습니다. 처음에는 학교 내의 학생들만을 대상으로 했지만, 이후 전 세계 수백만 명의 사용자를 보유한 글로벌 플랫폼으로 성장했습니다. Facebook은 사용자 데이터를 수집하고 이를 기반으로 타겟 광고를 제공하는 것으로 알려져 있으며, 이는 회사의 주요 수익원입니다.

Facebook은 개인정보 보호 문제와 잘못된 정보 확산에 대한 비판을 받아왔으며, 이로 인해 많은 논란이 일고 있습니다. 그럼에도 불구하고 전 세계적으로 가장 인기 있는 소셜 미디어 플랫폼 중 하나로 남아있으며, 사람들이 연결되고 정보를 공유하는 데 있어 중요한 역할을 하고 있습니다.

['\nYouTube는 사용자가 비디오를 업로드하고, 시청하며, 공유할 수 있는 인기있는 온라인 동영상 플랫폼입니다. 다양한 콘텐츠 범위가 있으며, 교육적인 것부터 오락적인 것에 이르기까지 다양합니다. 사용자는 자신의 채널을 만들고 구독자를 모으며 수익을 창출할 수도 있습니다.',
 '\n인스타그램은 사진 및 비디오 공유를 위한 소셜 미디어 플랫폼으로, 사용자들이 자신의 경험을 친구들과 팔로워들과 공유할 수 있게 해줍니다. 인스타그램의 주요 기능으로는 필터 적용, 위치 태깅, 해시태그가 있습니다. 또한 사용자가 다른 사용자의 콘텐츠와 상호 작용할 수 있도록 좋아요 버튼과 댓글 기능을 제공합니다.',
 '\nFacebook은 전 세계 수백만 명의 사람들이 친구, 가족 및 관심사와 연결될 수 있는 소셜 네트워킹 플랫폼입니다. 사용자들은 자신의 프로필을 만들고 업데이트를 게시하며 다른 사람들과 상호 작용할 수 있습니다. 또한 페이스북 메신저와 인스타그램과 같은 다양한 기능을 제공하여 사용자가 서로 소통하고 콘텐츠를 공유할 수 있게 합니다.\n\nFacebook은 2004년 하버드 대학교 학생인 마크 주커버그에 의해 창립되었습니다. 처음에는 학교 내의 학생들만을 대상으로 했지만, 이후 전 세계 수백만 명의 사용자를 보유한 글로벌 플랫폼으로 성장했습니다. Facebook은 사용자 데이터를 수집하고 이를 기반으로 타겟 광고를 제공하는 것으로 알려져 있으며, 이는 회사의 주요 수익원입니다.\n\nFacebook은 개인정보 보호 문제와 잘못된 정보 확산에 대한 비판을 받아왔으며, 이로 인해 많은 논란이 일고 있습니다. 그럼에도 불구하고 전 세계적으로 가장 인기 있는 소셜 미디어 플랫폼 중 하나로 남아있으며, 사람들이 연결되고 정보를 공유하는 데 있어 중요한 역할을 하고 있습니다.']

# LangChain Expression Language(LCEL)

https://wikidocs.net/233344

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

# 1
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

# 2
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

# 4
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = ChatPromptTemplate.from_template(template)


##########
# llm 모델
##########
local_path = (
    "./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"
)
llm = GPT4All(
    model=local_path,
    callbacks=[StreamingStdOutCallbackHandler()],
    backend="mps", # GPU 설정
    streaming=True,
    verbose=True,
)


chain = (
    prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))


영어 대화 예시:

English Conversation:
I would like to order some food at the restaurant, please.
Can I have a menu?
Sure, here you go. Please take your time and look through it. If you need any help or recommendations, just let me know.
Thank you very much! I'll take a moment to decide what I want.
Great, we appreciate your business. Let us know when you're ready to place your order.
I think I found something that looks good. Can I have the [specific dish] please?
Sure thing! Would you like anything else with it or any side dishes?
No thank you, just the main dish is fine for now.
Alright then, we will get right on that and bring it out to your table shortly. Enjoy your meal!
영어 대화 예시:

English Conversation:
I would like to order some food at the restaurant, please.
Can I have a menu?
Sure, here you go. Please take your time and look through it. If you need any help or recommendations, just let me know.
Thank you very much! I'll take a moment to decide what I want.
Great, we appreciate you

# JSONPatch 청크 스트리밍

https://wikidocs.net/233345

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
##########
# llm 모델
##########
local_path = (
    "./EEVE-Korean-Instruct-10.8B-v1.0-gguf/EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"
)
llm = GPT4All(
    model=local_path,
    callbacks=[StreamingStdOutCallbackHandler()],
    backend="mps", # GPU 설정
    streaming=True,
    verbose=True,
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
#################
# embedding model
#################
model_name = 'BM-K/KoSimCSE-roberta'
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True}

embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
###################################
#  FAISS 벡터 스토어를 이용해 텍스트를 인덱싱
###################################
vectorstore = FAISS.from_texts(
    ["원영이가 살고 있는 곳은 대한민국 입니다."], embedding=embeddings_model
)
#########################################################################
# retriever와 RunnablePassthrough를 통해 질문의 컨텍스트를 검색하고, 질문을 그대로 전달
# retrieval_chain은 이러한 컴포넌트들을 파이프라인으로 연결하고, StrOutputParser를 사용하여 모델의 출력을 문자열로 파싱합니다.
#########################################################################
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다
async for chunk in retrieval_chain.astream_log(
    "원영이가 살고 있는 곳은 어딘가요?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with MEAN pooling.
/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/langchain/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '65842086-4c8f-4148-9f4c-0d58101b8f59',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'ecfbd609-acec-4293-80ea-609f46f99ad9',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-05-14T13:39:57.127+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'HuggingFaceBgeEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='원영이가 살고 있는 곳은 대한민국 입니다.')

In [2]:
# retrieval_chain 객체의 astream_log 비동기 메서드를 사용하여 로그 데이터를 비동기적으로 검색
# "Docs"를 포함한 로그만을 필터링합니다. diff 매개변수는 False로 설정되어 있어, 변경 사항에 대한 차이점을 표시하지 않습니다. 
# 각 로그 청크(chunk)가 검색될 때마다, 청크의 내용을 출력하기 전에 구분선을 출력합니다

async for chunk in retrieval_chain.astream_log(
    "원영가 살고 있는 곳은 어딘가요?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)


----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1256bc89-d924-4cef-99e4-6060f52f153a',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1256bc89-d924-4cef-99e4-6060f52f153a',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'ba928e32-0711-47d7-b160-d48ca6c56e32',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-05-14T13:40:09.790+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context',
                            'FAISS',
                            'HuggingFaceBgeEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'

In [3]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야?") | llm

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = ChatPromptTemplate.from_template("{country} 의 면적은 얼마야?") | llm
# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)


In [4]:
chain1.invoke({"country": "대한민국"})


답변: 서울입니다.
신뢰도: 95%

'\n답변: 서울입니다.\n신뢰도: 95%'

In [5]:
chain2.invoke({"country": "대한민국"})


대한민국의 총면적이 몇 제곱킬로미터인지 알고 싶어요.

'\n대한민국의 총면적이 몇 제곱킬로미터인지 알고 싶어요.'

In [6]:
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])


답변: 서울입니다.
신뢰도: 95%
신뢰도: 95%

['\n답변: 서울입니다.\n신뢰도: 95%', '\n신뢰도: 95%']

In [7]:
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])


대한민국의 총면적이 몇 제곱킬로미터인지 알고 싶어요.
미국 대륙의 총 면적을 말씀하시는 건가요, 아니면 미국의 영토를 말하는 건지요? 만약 후자라면 983만 제곱킬로미터(약 379만 평방마일)입니다.

['\n대한민국의 총면적이 몇 제곱킬로미터인지 알고 싶어요.',
 '\n미국 대륙의 총 면적을 말씀하시는 건가요, 아니면 미국의 영토를 말하는 건지요? 만약 후자라면 983만 제곱킬로미터(약 379만 평방마일)입니다.']

In [7]:
# 주어진 'country'에 대해 'combined' 객체의 'invoke' 메서드를 호출합니다.
combined.invoke({"country": "대한민국"})

: 

In [8]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

: 